# Add helper function to show images in a matrix

In [ ]:
import os
from matplotlib.pyplot import figure, imshow, axis
from matplotlib import pyplot as plt
from matplotlib.image import imread

hSize = 300
wSize = 300
def showImagesMatrix(images_path = None, list_of_image_files = None, list_of_image_titles = None, col=10):
    if not list_of_image_files:
        list_of_image_files = os.listdir(images_path)
    fig = figure( figsize=(wSize, hSize))
    number_of_files = len(list_of_image_files)
    row = number_of_files/col
    if (number_of_files%col != 0):
        row += 1
    for i in range(number_of_files):
        a=fig.add_subplot(row,col,i+1)
        if images_path:
            image = imread(images_path+'/'+list_of_image_files[i])
        else:
            image = list_of_image_files[i]
        if list_of_image_titles:
            plt.title(list_of_image_titles[i])
        imshow(image,cmap='Greys_r')
        axis('off')

# Add helper function to calculate camera calibration coefficients

In [ ]:
import numpy as np
import cv2
import matplotlib.image as mpimg

chessboard_height = 6
chessboard_width = 9

def calculate_camera_cal_coffs(images_path):
    objpoints = [] # 3D points in real world space
    imgpoints = [] # 2D points in image plane

    images_with_corners = []

    chessboard_height = 6
    chessboard_width = 9

    # Prepae object points, like (0,0,0), (1,0,0), (2,0,0) ..., (7,5,0)
    objp = np.zeros((chessboard_height*chessboard_width,3), np.float32)
    objp[:,:2] = np.mgrid[0:chessboard_width,0:chessboard_height].T.reshape(-1, 2) #x,y coordinates

    for fname in images_path:
        # read in each image
        img = mpimg.imread(fname)

        # convert image into gray scale
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (chessboard_width,chessboard_height), None)

        # If corners are found, add object points, image points
        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp)

            # draw abd dusolay the coreners
            img = cv2.drawChessboardCorners(img, (chessboard_width,chessboard_height), corners, ret)
            images_with_corners.append(img)
    return imgpoints, objpoints, images_with_corners

In [ ]:
import glob 

# Read in and make a list of calibaration images
images_path = glob.glob('./camera_cal/calibration*.jpg')

imgpoints, objpoints, images_with_corners = calculate_camera_cal_coffs(images_path)
showImagesMatrix(list_of_image_files = images_with_corners, list_of_image_titles = images_path, col=2)

# Add helper function to undistort the image

In [ ]:
def cal_undistort(img, objpoints, imgpoints):
    # convert image into gray scale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

# Undistort one of the calibration images

In [ ]:
# read in one calibration image
img = mpimg.imread('./camera_cal/calibration1.jpg')

undist = cal_undistort(img, objpoints, imgpoints)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undist)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Undistort one sample of the calibration images and test images

In [ ]:
# read in one test image
cal_img = mpimg.imread('./camera_cal/calibration1.jpg')
cal_undist = cal_undistort(cal_img, objpoints, imgpoints)
print(cal_img.shape[::-1])
test_img = mpimg.imread('./test_images/test2.jpg')
test_undist = cal_undistort(test_img, objpoints, imgpoints)


f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(cal_img)
ax1.set_title('Original Caliration Image', fontsize=20)
ax2.imshow(cal_undist)
ax2.set_title('Undistorted Caliration Image', fontsize=20)

ax3.imshow(test_img)
ax3.set_title('Original Test Image', fontsize=20)
ax4.imshow(test_undist)
ax4.set_title('Undistorted Test Image', fontsize=20)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Add helper functions to undistort and unwarp the images

In [ ]:
src = np.float32([[520, 508], [770,508],[1030,687],[275,678]])
dst = np.float32([[275,508],[1030,508],[1030,678],[275,678]])

def corners_unwarp(img, nx, ny, mtx, dist):
    # Pass in your image into this function
    # Write code to do the following steps
    # 1) Undistort using mtx and dist
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    img_size = img.shape[1::-1]
    # 2) define 4 source points src = np.float32([[,],[,],[,],[,]])
         #Note: you could pick any four of the detected corners 
         # as long as those four corners define a rectangle
         #One especially smart way to do this would be to use four well-chosen
         # corners that were automatically detected during the undistortion steps
         #We recommend using the automatic detection of corners in your code
    #src = np.float32([[520, 508], [770,508],[1030,687],[275,678]])
    """src = np.float32(
    [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
    [((img_size[0] / 6) - 10), img_size[1]],
    [(img_size[0] * 5 / 6) + 60, img_size[1]],
    [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]]) """
    
    # 3) define 4 destination points dst = np.float32([[,],[,],[,],[,]])
    #dst = np.float32([[275,508],[1030,508],[1030,678],[275,678]])
    """dst = np.float32(
    [[(img_size[0] / 4), 0],
    [(img_size[0] / 4), img_size[1]],
    [(img_size[0] * 3 / 4), img_size[1]],
    [(img_size[0] * 3 / 4), 0]]) """
    # 4) use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    
    # 5) use cv2.warpPerspective() to warp your image to a top-down view
    img_size = img.shape[1::-1]
    warped = cv2.warpPerspective(undist, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, M

def cal_undistort_unwarp(img, objpoints, imgpoints):
    # convert image into gray scale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    nx = chessboard_width
    ny = chessboard_height
    warped, M = corners_unwarp(img, nx, ny, mtx, dist)
    return warped, M

# Undistort and unwarp the test images

In [ ]:
# Read in and make a list of calibaration images
test_images_path = glob.glob('./test_images/test*.jpg')
bird_eye_perspective_imgs = []
for fname in test_images_path:
    # read in each image
    img = mpimg.imread(fname)   
    top_down, perspective_M = cal_undistort_unwarp(img, objpoints, imgpoints)
    bird_eye_perspective_imgs.append(top_down)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(top_down)
    ax2.set_title('Undistorted and Warped Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# HLS Color and gradiant

In [ ]:
def color_gradient(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    r_channel = img[:,:,0]
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(r_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    #color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    binary_img = sxbinary | s_binary
    return binary_img

# Apply color_gradient for test images

In [ ]:
# Read in and make a list of test images
test_images_path = glob.glob('./test_images/test*.jpg')

for fname in test_images_path:
    # read in each image
    img = mpimg.imread(fname)   
    binary_img = color_gradient(img)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(binary_img, cmap = 'gray')
    ax2.set_title('color and gradient Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Apply HLS color and sobel gradient for bird eye perspective of test images

In [ ]:
bird_eye_gradient_imgs = []
for img in bird_eye_perspective_imgs:
    binary_img = color_gradient(img)
    bird_eye_gradient_imgs.append(binary_img)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(binary_img, cmap = 'gray')
    ax2.set_title('color and gradient Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Adding helper function to find histogram of the image

In [ ]:
def hist(img):
    # TO-DO: Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = img[img.shape[0]//2:, ...]
    # TO-DO: Sum across image pixels vertically - make sure to set `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    return histogram

# Visualizing histogram of the bird eye perspective of test images

In [ ]:
# Load our image
for img in bird_eye_gradient_imgs:
    # Create histogram of image binary activations
    histogram = hist(img)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img, cmap = 'gray')
    ax1.set_title('Original Image', fontsize=50)
    ax2.plot(histogram) # Visualize the resulting histogram
    ax2.set_title('histogram Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Implementing Sliding Windows and Fit a Polynomial

In [ ]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high) &\
            (nonzeroy >= win_y_low) & (nonzeroy < win_y_high)).nonzero()[0]
        good_right_inds = ((nonzerox >= win_xright_low) & (nonzerox < win_xright_high) &\
            (nonzeroy >= win_y_low) & (nonzeroy < win_y_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    return out_img, left_fitx, right_fitx, ploty, left_fit, right_fit

# Visualizing Sliding Windows and Fit a Polynomial for the bird eye perspective of test images

In [ ]:
# Load our image
left_fits = [] 
right_fits = []

for img in bird_eye_gradient_imgs:
    # Create histogram of image binary activations
    out_img, left_fitx, right_fitx, ploty, left_fit, right_fit = fit_polynomial(img)
    
    left_fits.append(left_fit)
    right_fits.append(right_fit)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img, cmap = 'gray')
    ax1.set_title('bird eye gradient Image', fontsize=50)
    
    ax2.imshow(out_img)
    # Plots the left and right polynomials on the lane lines
    ax2.plot(left_fitx, ploty, color='yellow')
    ax2.plot(right_fitx, ploty, color='yellow')
    
    ax2.set_title('Sliding Windows and Fit a Polynomial Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Finding the lane lines calculated by searching from prior detections

In [ ]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

def search_around_poly(binary_warped, left_fit, right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    # Identify window boundaries in x and y (and right and left)
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result, left_fitx, right_fitx, ploty

# Visualizing the lane lines calculated by searching from prior detections

In [ ]:
bird_eye_offset_lanes_imgs = []
for img, left_fit, right_fit in zip(bird_eye_gradient_imgs, left_fits, right_fits):
    # Create histogram of image binary activations
    result, left_fitx, right_fitx, ploty = search_around_poly(img, left_fit, right_fit)
    bird_eye_offset_lanes_imgs.append(result)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img, cmap = 'gray')
    ax1.set_title('bird eye gradient Image', fontsize=50)
    
    ax2.imshow(result)
    # Plots the left and right polynomials on the lane lines
    ax2.plot(left_fitx, ploty, color='yellow')
    ax2.plot(right_fitx, ploty, color='yellow')
    
    ax2.set_title('Sliding Windows and Fit a Polynomial Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)



# Measuring Curvature

In [ ]:
def measure_curvature_real(img):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(img)

    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

# Printing Measured Curvature values in meters

In [ ]:
for img in bird_eye_gradient_imgs:
    left_curverad, right_curverad = measure_curvature_real(img)
    f, (ax1) = plt.subplots(1, 1, figsize=(12, 4))
    f.tight_layout()
    ax1.imshow(img, cmap = 'gray')
    ax1.set_title(str('left_curverad: ' + str(left_curverad) + ', right_curverad: ' + str(right_curverad) + ', diff: ' + str(left_curverad - right_curverad)), fontsize=20)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    

# Implementing draw a green lane on the test images

In [ ]:
# use cv2.getPerspectiveTransform() to get M, the transform matrix
inverse_M = cv2.getPerspectiveTransform(dst, src)

def combine_img_with_green_lane(image, binary_warped, left_fit, right_fit):
    undist = cal_undistort(image, objpoints, imgpoints)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (inverse_M)
    newwarp = cv2.warpPerspective(color_warp, inverse_M, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR) 
    
    # Combine the result with the original image
    green_lane_img = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    return green_lane_img

# Draw a green lane on the test images

In [ ]:
for fname, binary_warped, left_fit, right_fit in zip(test_images_path, bird_eye_gradient_imgs, left_fits, right_fits): 
    # read in each image
    image = mpimg.imread(fname) 
    
    green_lane_img = combine_img_with_green_lane(image, binary_warped, left_fit, right_fit)
    
    left_curverad, right_curverad = measure_curvature_real(binary_warped)
    
    f, (ax1) = plt.subplots(1, 1, figsize=(12, 4))
    f.tight_layout()
    ax1.imshow(green_lane_img)
    
    ax1.set_title(str('left_curverad: ' + str(left_curverad) + ', right_curverad: ' + str(right_curverad) + ', diff: ' + str(left_curverad - right_curverad)), fontsize=20)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Define a line class to track the lfet and right lane lines data

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None 

# Implementing the complete pipe line in one function

In [ ]:
def process_image(image):
    #undistort the input image
    cal_undist = cal_undistort(image, objpoints, imgpoints)

    #undistort & unwarp [convert to bird eye perspective] the input image
    top_down_image, perspective_M = cal_undistort_unwarp(image, objpoints, imgpoints)

    # Apply HLS color and sobel gradient for bird eye perspective of test images
    top_down_binary_img = color_gradient(top_down_image)

    # Create histogram of image binary activations
    #the out_img is the top down binary image after adding sliding windows on its output and mark the left lane points as red and the right lane points as blue
    out_img, _, __, ___, left_fit_coffs, right_fit_coffs = fit_polynomial(top_down_binary_img)

    #Measure Curvature values in meters
    left_curverad, right_curverad = measure_curvature_real(top_down_binary_img)

    # Draw a green lane on the input image
    green_lane_img = combine_img_with_green_lane(image, top_down_binary_img, left_fit_coffs, right_fit_coffs)

    return green_lane_img

# Run the pipe line on the test images

In [ ]:
test_images_path = glob.glob('./test_images/test*.jpg')
for fname in test_images_path:
    # read in each image
    image = mpimg.imread(fname) 
    
    green_lane_img = process_image(image)
    
    left_curverad, right_curverad = measure_curvature_real(binary_warped)
    
    f, (ax1) = plt.subplots(1, 1, figsize=(12, 4))
    f.tight_layout()
    ax1.imshow(green_lane_img)
    
    ax1.set_title(str('left_curverad: ' + str(left_curverad) + ', right_curverad: ' + str(right_curverad) + ', diff: ' + str(left_curverad - right_curverad)), fontsize=20)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Run the pipline on a video input

In [ ]:
# Import everything needed to edit/save/watch video clips# Import 
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
yellow_output = 'output_videos/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))